In [1]:
#Functions for parsing columns of the spreadsheet on Santa Clara County School Meals
import re
abbrev_to_index = {
    'm': 0, 'mon': 0, 'monday': 0, 'mondays': 0,
    't': 1, 'tues': 1, 'tuesday': 1, 'tuesdays': 1,
    'w': 2, 'wed': 2, 'wednesday': 2, 'wednesdays': 2,
    'th': 3, 'thr': 3, 'thurs': 3, 'thursday': 3, 'thursdays': 3,
    'f': 4, 'fri': 4, 'friday': 4, 'fridays': 4,
    's': 5, 'sat': 5, 'saturday': 5, 'saturdays': 5, 'sa': 5,
    'su': 6, 'sun': 6, 'sunday': 6, 'sundays': 6
}

daysOfWeek = {
    '0': 'Monday',
    '1': 'Tuesday',
    '2': 'Wednesday',
    '3': 'Thursday',
    '4': 'Friday',
    '5': 'Saturday',
    '6': 'Sunday',

}


def parse_address(addr):
    regex = r"([\w|.| |\d|-]+)[,\s*]+([\w|\s]+)[,\s*]*(.*)[-| ]+(.*)"
    matches = re.finditer(regex, addr, re.IGNORECASE | re.DOTALL)
    address = [addr]
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(1, len(match.groups())+1):
            address.append(match.group(groupNum))
    return tuple(address)

def fcnparse_siteaddress(csvaddress):
    myoutput = nominatim.query(csvaddress+ ' California')
    if len(myoutput.toJSON())>0:
        myjson = myoutput.toJSON()[0]
    
        #print(myjson)
        #print(type(myjson))
        full_address = myjson['display_name']
        #print(full_address)
        #print(type(full_address))

        tmp = full_address.split(', ')
        #print(tmp)
       
        if tmp[0].isnumeric():
            address = tmp[0]+" "+tmp[1]
            city = tmp[-5:-4][0]
            state = tmp[-3:-2][0]
            zipcode = tmp[-2:-1][0]            
        elif tmp[1].isnumeric():
            address = tmp[1]+" "+tmp[2]
            city = tmp[-5:-4][0]
            state = tmp[-3:-2][0]
            zipcode = tmp[-2:-1][0]
        else:
            address = csvaddress+" ######CHECK ME######"
            city = "######CHECK ME######"
            zipcode = "######CHECK ME######"
            state = "California"
    else:
        full_address = ""
        address = csvaddress+"######CHECK ME######"
        city = "######CHECK ME######"
        zipcode = "######CHECK ME######"
        state = "California"
        
    return full_address, address, city, state, zipcode



def fcnparse_opendays_opentime(dh_str):
    dh_str = dh_str.replace("-", " to ")
    try:
        days = dh_str.split("@")[0].split("\n")[0].lower()

        regex = r"(?:\W+)[,|-|\s|&]*"
        l = re.compile(regex).split(days)
        matches = re.finditer(regex, days)

        _days=[]
        new_days = daysOfWeek[str(abbrev_to_index[l[0]])] if l[0] in abbrev_to_index.keys() else l[0]
        if l[0] in abbrev_to_index.keys():
            _days.append(daysOfWeek[str(abbrev_to_index[l[0]])])

        for matchNum, match in enumerate(matches, start=1):
            if l[matchNum] in ["to", "-", "through"]:
                startDay = abbrev_to_index[l[matchNum - 1]] + 1
                endDay = abbrev_to_index[l[matchNum + 1]]
                for i in range(startDay, endDay):
                    _days.append(daysOfWeek[str(i)])
            else:
                if l[matchNum] in abbrev_to_index.keys():
                    _days.append( daysOfWeek[str(abbrev_to_index[l[matchNum]])])

        days= ",".join(list(set(_days)))

        timing = dh_str.split("@")[1]
        timing = re.findall("(\d{2}:\d{2}\s*[aAPpmM]*)",timing)

        return (days, timing[0], timing[1] if len(timing)>1 else "")

    except:
        return "####### CHECK ME ########"



def fcnparse_mealtypes(attributesMEALTYPES):
    #print(attributesMEALTYPES)
    mealoptions = []
    if re.search('breakfast', attributesMEALTYPES, re.IGNORECASE):
        mealoptions = mealoptions + ["Breakfast", ]
    if re.search('lunch', attributesMEALTYPES, re.IGNORECASE):
        mealoptions = mealoptions + ["Lunch", ]  
    if re.search('dinner', attributesMEALTYPES, re.IGNORECASE):
        mealoptions = mealoptions + ["Dinner", ]   

    #print(mealoptions)
    return mealoptions


In [2]:
fcnparse_opendays_opentime('s,M,Th@11:15 am until 12:15 pm')

('Monday,Thursday,Saturday', '11:15 am', '12:15 pm')

In [3]:
import urllib

def getOrCreateAirTable(_url, _header, _data, _uniqueFieldName):
    _f ={
        "maxRecords": 1,
        "filterByFormula": "{"+_uniqueFieldName+"}='"+_data["fields"][_uniqueFieldName]+"'"
    }
    _filter = "?"+urllib.parse.urlencode(_f)
    _res = requests.get(_url+_filter, headers = _header)
    _recs = json.loads(_res.text.encode('utf8'))["records"]
    if len(_recs) == 0:
        return insertIntoAirTable(_url, _header, _data)
    else:
        _id = json.loads(_res.text.encode('utf8'))["records"][0]["id"]
        print("Existing Id -> "+str(_id) + "::: "+ _url)
        return _id
    
    
def insertIntoAirTable(_url, _header, _data):
    print("Insered Data -> {0}".format(_data))
    print ("."*40)
    _res = requests.post(_url, headers = _header, data = json.dumps(_data))
    print(_res.text)
    _id = json.loads(_res.text.encode('utf8'))["id"]
    print("Id -> "+str(_id) + "::: "+ _url)
    print("Response     -> {0}".format(_res.text))
    print("-"*80)
    return _id

def updateServicesAirTable(_url, _header, _data):
#     print("Update Data : "+ json.dumps(_data))
    _res = requests.patch(_url, headers = _header, data = json.dumps(_data))
#     print("Update Services "+ " ::: "+ _url)
#     print(_res.text)
#     print("*"*80)
    

In [ ]:
#Code to scrape Santa Clara County school meal sites
import requests
import csv
import json
import time
#from OSMPythonTools.nominatim import Nominatim
#nominatim = Nominatim()

post_url_services = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/services'
post_url_organizations = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/organizations'
post_url_address = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/address'
post_url_location = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/locations'
post_url_schedule = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/schedule'
post_url_phones = 'https://api.airtable.com/v0/applQOkth8R2ns3qo/phones'

post_headers = {
    'Authorization' : 'Bearer keyLBTgL8hjaceVN4',
    'Content-Type': 'application/json'
}
csvfilename = open('C:\deeya\code for sanjose\cfsj_csvtoairtable\Santa Clara Scrape 10_1 - Food Senior Meal Sites.csv', encoding='utf-8')
csv_f = csv.reader(csvfilename)

#Skip header row
count = 1
header= {}
col_num = 0
for row in csv_f:
    for f in row:
        header[f] = col_num
        col_num += 1

    break

#Loop over all rows and parse out the columns in each row
count = 1
for row in csv_f:
#     # skip till you find the Service to start at...
#     if row[5] != "Lakeside School":
#         continue
        
    
        
    rowcount = 0
    for elem in row:
        rowcount = rowcount+1
    #################################################
    #Upload to Service Table
    #################################################
    #Parse data for SERVICES sheet: attributes.SITENAME, attributes.MEALTYPES, 
    #attributes.SITESTATUS, attributes.DESCRIPTION, attributes.EMAIL, attributes.WEBLINK
    _temp = row[header["attributes.SITENAME"]].split(":")
    
    attributesSITENAME = _temp[1] if len(_temp)>1 else _temp[0]
    attributesMEALTYPES = fcnparse_mealtypes(row[header["attributes.MEALTYPES"]])
    attributesSITESTATUS = row[header["attributes.SITESTATUS"]]
    attributesDESCRIPTION = row[header["attributes.DESCRIPTION"]]+" "+row[header["attributes.DESCRIPTIONLONG"]]    
    attributesEMAIL = row[header["attributes.EMAIL"]]
    attributesWEBLINK = row[header["attributes.WEBLINK"]]
    airtableSERVICES = {
        "fields": {
            "Name": attributesSITENAME, 
            "active": attributesSITESTATUS,
            "Description": attributesDESCRIPTION,
            "email": attributesEMAIL,
            "url": attributesWEBLINK
        }
    }
    print("Upload to service table -> " + attributesSITENAME)
    serviceId = getOrCreateAirTable(post_url_services, post_headers, airtableSERVICES, "Name")
    
    
    ##################################################
    #Upload to Organization Table
    ##################################################
    attributesSITEADDRESS = row[header["attributes.SITEADDRESS"]]
    full_address, address, city, state, zipcode = parse_address(attributesSITEADDRESS)
        
    airtableORGANIZATIONS = {
        "fields": {
            "name": row[header["attributes.SITETYPE"]], 
            "description": row[header["attributes.DESCRIPTION"]]
        }
    }
    print("Upload to organization table")
    organizationId = getOrCreateAirTable(post_url_organizations, post_headers, airtableORGANIZATIONS, "name")
    
    ###################################################
    #Upload to Address Table
    ###################################################
    #Parse data for ADDRESS sheet: attributes.SITEADDRESS   
    #print(attributesSITEADDRESS)
    #full_address, address, city, state, zipcode = fcnparse_siteaddress(attributesSITEADDRESS)
    
        
    #print(count+1, address, city, state, zipcode, '\n')
    
#     address = "1234 Memory Lane"
#     city = "San Jose"
#     state = "CA"
#     zipcode = "95124"
    
    
    airtableADDRESS = {
        "fields": {
            "address_1": address,
            "city": city,
            "State": state,
            "Zip Code": zipcode
        }
    }
    print("Upload to address table")
    addressId = getOrCreateAirTable(post_url_address, post_headers, airtableADDRESS, "address_1")
    
    ####################################################
    # Upload to Phone Table
    ####################################################
    # Parse data for PHONES sheet: attributes.PHONENUMBER
    #attributesPHONENUMBER = row[14] if row[14] != '' else "n/a"
    airtablePHONES = {
        "fields": {
            "number": row[header["attributes.PHONENUMBER"]], 
            "services": [serviceId],
            "organizations": [organizationId] 
        }
    }
    print("Upload to phone table")
    phoneId = getOrCreateAirTable(post_url_phones, post_headers, airtablePHONES, "number")
    
    ##################################################
    #Upload to Schedule Table
    ##################################################
    #Parse data for SCHEDULE sheet: attributes.OPENDAYS, attributes.OPENTIMES
    attributesOPENDAYS = row[header["attributes.OPENDAYS"]]
    attributesOPENTIMES = row[header["attributes.OPENTIMES"]]
    opendaystime = fcnparse_opendays_opentime(attributesOPENDAYS+ '@'+ attributesOPENTIMES)
    airtableSCHEDULE = {
        "fields": { 
            "name": row[header["attributes.SITENAME"]],
            "services": [serviceId], 
            "weekday": opendaystime[0].split(","), 
            "opens_at": opendaystime[1], 
            "closes_at": opendaystime[2]  
        }
    } 
    
    if airtableSCHEDULE["fields"]["weekday"] == ['']:
        airtableSCHEDULE["fields"]["description-x"]  = "####### Please check data ######" 
        airtableSCHEDULE["fields"]["weekday"] = []
    
    print("Upload to schedule table")
    scheduleId = insertIntoAirTable(post_url_schedule, post_headers, airtableSCHEDULE)
   
    ###################################################
    #Upload to Location Table
    ###################################################
    airtableLOCATION = {
                          "fields": {
                            "name": row[header["attributes.SITENAME"]],
                            "latitude": row[header["geometry.x"]],
                            "longitude": row[header["geometry.y"]],
                            "services": [
                              serviceId
                            ],
                            "schedule": [
                              scheduleId
                            ],
                            "address": [
                              addressId
                            ]
                          }
                        }
    print("Upload to location table")
    LocationId = getOrCreateAirTable(post_url_location, post_headers, airtableLOCATION, "name")
    
    
    ####################################################
    # Update Services Table
    ####################################################
    
    airtableSERVICES = {
                          "fields": {
                            "Organization": [
                              organizationId
                            ],
                            "phones": [
                              phoneId
                            ],
                            "schedule":[
                              scheduleId
                            ],
                            "locations":[
                                LocationId
                            ]
                          }
                        }
    updateServicesAirTable("https://api.airtable.com/v0/applQOkth8R2ns3qo/services/"+serviceId, post_headers, airtableSERVICES)
    
    
    count = count + 1
#     if count == 4:
#         break    

Upload to service table -> Asian Americans for Community Involvement (AACI)
Insered Data -> {'fields': {'Name': 'Asian Americans for Community Involvement (AACI)', 'active': 'Open', 'Description': 'The Senior Nutrition Program is open to all seniors over the age of 60. A contribution of $3.00 is asked from each person to help offset the cost of the meal, however no eligible senior shall be denied participation because of inability to contribute. ', 'email': 'Gerard.Manuel@aaci.org', 'url': 'N/A'}}
........................................
{"id":"recuRhpc13dLlgtsI","fields":{"Name":"Asian Americans for Community Involvement (AACI)","Description":"The Senior Nutrition Program is open to all seniors over the age of 60. A contribution of $3.00 is asked from each person to help offset the cost of the meal, however no eligible senior shall be denied participation because of inability to contribute. ","url":"N/A","email":"Gerard.Manuel@aaci.org","id":9262,"active":"Open","Auto-Last updated":"2

Insered Data -> {'fields': {'name': 'Other Community Center', 'description': 'The Senior Nutrition Program is open to all seniors over the age of 60. A contribution of $3.00 is asked from each person to help offset the cost of the meal, however no eligible senior shall be denied participation because of inability to contribute.'}}
........................................
{"id":"recx0LAwf7cpmdbQA","fields":{"name":"Other Community Center","description":"The Senior Nutrition Program is open to all seniors over the age of 60. A contribution of $3.00 is asked from each person to help offset the cost of the meal, however no eligible senior shall be denied participation because of inability to contribute.","id":8859},"createdTime":"2020-11-15T19:13:50.000Z"}
Id -> recx0LAwf7cpmdbQA::: https://api.airtable.com/v0/applQOkth8R2ns3qo/organizations
Response     -> {"id":"recx0LAwf7cpmdbQA","fields":{"name":"Other Community Center","description":"The Senior Nutrition Program is open to all seniors

{"id":"recVQxU1cY8tVTDsA","fields":{"name":"Campbell Adult Center","latitude":"-13575408.39","longitude":"4479203.662","services":["recN6LOOh412aZcMp"],"schedule":["recGmV1a380PwhDSj"],"address":["recXnoRlsfYbl8SFa"],"id":5662,"address_1 (from address)":["1 W Campbell Ave"],"city (from address)":["Campbell"],"Zip Code (from address)":["95008"]},"createdTime":"2020-11-15T19:13:57.000Z"}
Id -> recVQxU1cY8tVTDsA::: https://api.airtable.com/v0/applQOkth8R2ns3qo/locations
Response     -> {"id":"recVQxU1cY8tVTDsA","fields":{"name":"Campbell Adult Center","latitude":"-13575408.39","longitude":"4479203.662","services":["recN6LOOh412aZcMp"],"schedule":["recGmV1a380PwhDSj"],"address":["recXnoRlsfYbl8SFa"],"id":5662,"address_1 (from address)":["1 W Campbell Ave"],"city (from address)":["Campbell"],"Zip Code (from address)":["95008"]},"createdTime":"2020-11-15T19:13:57.000Z"}
--------------------------------------------------------------------------------
Upload to service table ->  Berryessa Comm

Existing Id -> recKsDLA0kkElxJVu::: https://api.airtable.com/v0/applQOkth8R2ns3qo/organizations
Upload to address table
Existing Id -> recrgcjxDNuyBbqFV::: https://api.airtable.com/v0/applQOkth8R2ns3qo/address
Upload to phone table
Insered Data -> {'fields': {'number': '408-371-2888', 'services': ['recbsNyUx8aUsdLV3'], 'organizations': ['recKsDLA0kkElxJVu']}}
........................................
{"id":"recPi5XTDvsqOhrog","fields":{"number":"408-371-2888","services":["recbsNyUx8aUsdLV3"],"organizations":["recKsDLA0kkElxJVu"]},"createdTime":"2020-11-15T19:14:04.000Z"}
Id -> recPi5XTDvsqOhrog::: https://api.airtable.com/v0/applQOkth8R2ns3qo/phones
Response     -> {"id":"recPi5XTDvsqOhrog","fields":{"number":"408-371-2888","services":["recbsNyUx8aUsdLV3"],"organizations":["recKsDLA0kkElxJVu"]},"createdTime":"2020-11-15T19:14:04.000Z"}
--------------------------------------------------------------------------------
Upload to schedule table
Insered Data -> {'fields': {'name': 'CSJ: Camde

Upload to service table ->  Willow Glen Senior Center
Insered Data -> {'fields': {'Name': ' Willow Glen Senior Center', 'active': 'Open', 'Description': 'The Senior Nutrition Program is open to all seniors over the age of 60. A contribution of $3.00 is asked from each person to help offset the cost of the meal, however no eligible senior shall be denied participation because of inability to contribute. ', 'email': 'alvaro.celaya@sanjoseca.gov', 'url': 'N/A'}}
........................................
{"id":"recXAxofIMf6ZlG9r","fields":{"Name":" Willow Glen Senior Center","Description":"The Senior Nutrition Program is open to all seniors over the age of 60. A contribution of $3.00 is asked from each person to help offset the cost of the meal, however no eligible senior shall be denied participation because of inability to contribute. ","url":"N/A","email":"alvaro.celaya@sanjoseca.gov","id":9268,"active":"Open","Auto-Last updated":"2020-11-15T19:14:10.000Z","Auto-Last updated by":{"id":"u

...

{"id":"recxrPLrpzZV8ivOb","fields":{"name":"Santa Clara Valley Blind (Vista Center)","services":["rectYRE706ufowY2u"],"opens_at":"12:00 pm","closes_at":"12:30 pm","weekday":["Wednesday"],"id":3904},"createdTime":"2020-11-15T19:15:16.000Z"}
Id -> recxrPLrpzZV8ivOb::: https://api.airtable.com/v0/applQOkth8R2ns3qo/schedule
Response     -> {"id":"recxrPLrpzZV8ivOb","fields":{"name":"Santa Clara Valley Blind (Vista Center)","services":["rectYRE706ufowY2u"],"opens_at":"12:00 pm","closes_at":"12:30 pm","weekday":["Wednesday"],"id":3904},"createdTime":"2020-11-15T19:15:16.000Z"}
--------------------------------------------------------------------------------
Upload to location table
Insered Data -> {'fields': {'name': 'Santa Clara Valley Blind (Vista Center)', 'latitude': '-13573633.56', 'longitude': '4484611.423', 'services': ['rectYRE706ufowY2u'], 'schedule': ['recxrPLrpzZV8ivOb'], 'address': ['recYCLztHQSRp49P2']}}
........................................
{"id":"rechaqJow8O1KEFjW","fields":

Existing Id -> recx0LAwf7cpmdbQA::: https://api.airtable.com/v0/applQOkth8R2ns3qo/organizations
Upload to address table
Existing Id -> recTFSeEfmrfdt2UR::: https://api.airtable.com/v0/applQOkth8R2ns3qo/address
Upload to phone table
Insered Data -> {'fields': {'number': '408-297-4979', 'services': ['recUgXiPR9559LJ5o'], 'organizations': ['recx0LAwf7cpmdbQA']}}
........................................
{"id":"recjul9YwVWJxxTkv","fields":{"number":"408-297-4979","services":["recUgXiPR9559LJ5o"],"organizations":["recx0LAwf7cpmdbQA"]},"createdTime":"2020-11-15T19:15:24.000Z"}
Id -> recjul9YwVWJxxTkv::: https://api.airtable.com/v0/applQOkth8R2ns3qo/phones
Response     -> {"id":"recjul9YwVWJxxTkv","fields":{"number":"408-297-4979","services":["recUgXiPR9559LJ5o"],"organizations":["recx0LAwf7cpmdbQA"]},"createdTime":"2020-11-15T19:15:24.000Z"}
--------------------------------------------------------------------------------
Upload to schedule table
Insered Data -> {'fields': {'name': 'Yu Ai Kai'

{"id":"rec6BuqDNx7h1lO00","fields":{"Name":"Sunnyvale Senior Nutrition (First United Methodist Church)","Description":"The Senior Nutrition Program is open to all seniors over the age of 60. A contribution of $3.00 is asked from each person to help offset the cost of the meal, however no eligible senior shall be denied participation because of inability to contribute. ","url":"N/A","id":9288,"active":"Open","Auto-Last updated":"2020-11-15T19:15:30.000Z","Auto-Last updated by":{"id":"usrh411cdZHMZYeMq","email":"23deeyav@students.harker.org","name":"Deeya Viradia"}},"createdTime":"2020-11-15T19:15:30.000Z"}
Id -> rec6BuqDNx7h1lO00::: https://api.airtable.com/v0/applQOkth8R2ns3qo/services
Response     -> {"id":"rec6BuqDNx7h1lO00","fields":{"Name":"Sunnyvale Senior Nutrition (First United Methodist Church)","Description":"The Senior Nutrition Program is open to all seniors over the age of 60. A contribution of $3.00 is asked from each person to help offset the cost of the meal, however no 

Insered Data -> {'fields': {'name': 'India Community Center (Milpitas)', 'latitude': '-13570690.75', 'longitude': '4498854.492', 'services': ['recxtGCPYFrW4v5CS'], 'schedule': ['recAFQDGjhxuMYaTl'], 'address': ['recr51ovWVoYofKim']}}
........................................
{"id":"recyVCU8Hy4i9lUXe","fields":{"name":"India Community Center (Milpitas)","latitude":"-13570690.75","longitude":"4498854.492","services":["recxtGCPYFrW4v5CS"],"schedule":["recAFQDGjhxuMYaTl"],"address":["recr51ovWVoYofKim"],"id":5687,"address_1 (from address)":["525 Los Coches St"],"city (from address)":["Milpitas"],"Zip Code (from address)":["95035"]},"createdTime":"2020-11-15T19:15:37.000Z"}
Id -> recyVCU8Hy4i9lUXe::: https://api.airtable.com/v0/applQOkth8R2ns3qo/locations
Response     -> {"id":"recyVCU8Hy4i9lUXe","fields":{"name":"India Community Center (Milpitas)","latitude":"-13570690.75","longitude":"4498854.492","services":["recxtGCPYFrW4v5CS"],"schedule":["recAFQDGjhxuMYaTl"],"address":["recr51ovWVoYofK

Existing Id -> recic1FE2RYnFYExU::: https://api.airtable.com/v0/applQOkth8R2ns3qo/organizations
Upload to address table
Existing Id -> recpa6Nqu7KtJoDlT::: https://api.airtable.com/v0/applQOkth8R2ns3qo/address
Upload to phone table
Insered Data -> {'fields': {'number': '408-354-4782', 'services': ['recsjYdwYFp7xDYIc'], 'organizations': ['recic1FE2RYnFYExU']}}
........................................
{"id":"recETk1lUUP9ZcxRt","fields":{"number":"408-354-4782","services":["recsjYdwYFp7xDYIc"],"organizations":["recic1FE2RYnFYExU"]},"createdTime":"2020-11-15T19:15:43.000Z"}
Id -> recETk1lUUP9ZcxRt::: https://api.airtable.com/v0/applQOkth8R2ns3qo/phones
Response     -> {"id":"recETk1lUUP9ZcxRt","fields":{"number":"408-354-4782","services":["recsjYdwYFp7xDYIc"],"organizations":["recic1FE2RYnFYExU"]},"createdTime":"2020-11-15T19:15:43.000Z"}
--------------------------------------------------------------------------------
Upload to schedule table
Insered Data -> {'fields': {'name': 'Live Oak A

In [95]:
#Prototyping code for parsing addresses

from OSMPythonTools.nominatim import Nominatim
nominatim = Nominatim()
#myoutput = nominatim.query('1680 Foley ave')
#print(myoutput.toJSON())
csvaddress = '592 Dunholme Way, san jose'
myoutput = nominatim.query(csvaddress)
print(myoutput)
print(myoutput.toJSON())
print(len(myoutput.toJSON()))
myjson = myoutput.toJSON()[0]
print(myjson)
print(type(myjson))
full_address = myjson['display_name']
print(full_address)
print(type(full_address))

tmp = full_address.split(', ')
print(tmp)
firstelem = tmp[0]
print(firstelem)
print(firstelem.isnumeric())
if tmp[0].isnumeric():
    address = tmp[0]+" "+tmp[1]
    city = tmp[-5:-4][0]
    state = tmp[-3:-2][0]
    zipcode = tmp[-2:-1][0]            
elif tmp[1].isnumeric():
    address = tmp[1]+" "+tmp[2]
    city = tmp[-5:-4][0]
    state = tmp[-3:-2][0]
    zipcode = tmp[-2:-1][0]
else:
    address = csvaddress+" ######CHECK ME######"
    city = "######CHECK ME######"
    zipcode = "######CHECK ME######"
    state = "California"

print(address)
print(city)
print(state)
print(zipcode)

ModuleNotFoundError: No module named 'OSMPythonTools'

In [87]:
if ~len([]):
    print("yup empty")

yup empty


In [123]:
#blah = '1680'
#blah.isdigit()
print(1!=2)

True


In [11]:
#Prototyping code for parsing meal types

import re

if re.search('mandye', 'Mandy Pande', re.IGNORECASE):
    print("Yes found")
    
test_tup1 = [1, 3, 5] 
test_tup2 = [4,]
  
# printing original tuples 
print("The original tuple 1 : " + str(test_tup1)) 
print("The original tuple 2 : " + str(test_tup2)) 
  
# Ways to concatenate tuples 
# using + operator 
res = test_tup1 + test_tup2 
  
# printing result 
print("The tuple after concatenation is : " + str(res))

The original tuple 1 : [1, 3, 5]
The original tuple 2 : [4]
The tuple after concatenation is : [1, 3, 5, 4]
